In [1]:
%matplotlib inline

import numpy as np
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import seaborn as sns
import matplotlib.pyplot as plt

import glob

import subprocess
from subprocess import call

import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas

import pickle

from scipy import stats

import numpy as np
from scipy.spatial.distance import cdist


Bad key text.latex.unicode in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://githu

In [2]:
# open SET1 (PhyloSET) and SET2 (RiboSET)
PhyloSET = pd.read_csv('../tmp_res/SET1.txt', sep='\t')



RiboSET = pd.read_csv('../tmp_res/SET2.txt', sep='\t')

RiboSET['tr_id1'] = [x.split('.')[0] for x in RiboSET['tr_id'].tolist()]

RiboSET = RiboSET[(~RiboSET['Riboseq_Summary'].str.contains('ATG')) & 
                  (~RiboSET['tr_id'].str.contains('PAR'))]

print (RiboSET.shape[0])

390


In [3]:
metadata_pc_g25 = pd.read_csv('../tmp_res/metadata_pc_g25.txt', sep='\t')
metadata_pc_g25[0:1]

,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
0,ENST00000335137.3,OR4F5,OR4F5-001,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,NaN,ATG,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,ATG,TAG,0,918,0,0,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...


In [4]:
# open metadata with scores and metrics
meta = pd.read_csv('../tmp_res/local_and_global_df_g25ovlp_g35ovlp_g38ovlp_refseqovlp_strand_FRAMES_PhyloCSF.txt', sep='\t')
meta[0:1]

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,max_sum_overlap_refseq_all_exons,max_sum_overlap_refseq_all_exons_strandless,ovlp,ovlp_strandless,ovlp2,ovlp2_strandless,PhyloCSF120score,tr_id1,number_of_records,records_names
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-163.9926,ENST00000624431,85,Human; Gorilla; Orangutan; Golden_snub_nosed_m...


In [5]:
RiboSET_EXT_cds_cov_nonzero = pd.read_csv('jupyters/FINAL_RiboSET_EXT_cds_cov_nonzero_theor_len_20codons_3532genes.txt', 
                                  sep='\t')


print (RiboSET_EXT_cds_cov_nonzero.shape[0])
RiboSET_EXT_cds_cov_nonzero[0:2]

3451


,tr_id1,Z_score,CDS_coverage,NTE_coverage,CDS_nonzero_fraction,NTE_nonzero_fraction,gene_ext,Start_ext,Stop_ext,global_rank_ext,...,max_sum_overlap_g38_all_exons_strandless,max_sum_overlap_refseq_all_exons,max_sum_overlap_refseq_all_exons_strandless,ovlp,ovlp_strandless,ovlp2,ovlp2_strandless,PhyloCSF120score,number_of_records,records_names
0,ENST00000379454,0.032101,298.318841,317.934783,0.332894,0.260870,ASPH,144,189,1,...,0,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-1473.5792,93,Human; Chimp; Bonobo; Gorilla; Orangutan; Gibb...
1,ENST00000216281,0.139750,195.778990,441.650000,0.158709,0.194444,HSP90AA1,168,207,2,...,0,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-1331.1851,63,Human; Chimp; Gorilla; Orangutan; Gibbon; Crab...


# Genomic coo of theoretical extensions for PhyloSET+RiboSET_ext 

In [6]:
PhyloSET[['tr_id', 'gene', 'global_coo_primary', 'strand']][0:2]

,tr_id,gene,global_coo_primary,strand
0,ENST00000377898.3,HES3,chr1:6244194-6244241+chr1:6244351-6244365,+
1,ENST00000400907.6,RERE,chr1:8656298-8656369,-


In [7]:
# prepare bed file and sort it 

def get_bed(filepath, df):
    '''
    df: tr_id, strand, coo
    '''
    f = open(filepath, 'w')

    for el in df.to_numpy():
        coo = el[2].split('+')
        strand = el[1]
        tr_id = el[0]
    
        for coo_ in coo:
            chrom = coo_.split(':')[0]
            start = coo_.split(':')[1].split('-')[0]
            stop = coo_.split(':')[1].split('-')[1]
        
        
            if int(start) - int(stop) == 1:
                f.write(chrom+'\t'+start+'\t'+str(int(stop)+1)+'\t'+tr_id+'\t1\t'+strand+'\n') 
                # where the length of the region is 0 nt
            else:
                f.write(chrom+'\t'+start+'\t'+stop+'\t'+tr_id+'\t1\t'+strand+'\n')
            
    f.close()  

In [8]:
get_bed(filepath='PhyloSET_genomic_primary_ext.bed', 
        df=PhyloSET[['tr_id', 'strand', 
                                'global_coo_primary']])

In [9]:
get_bed(filepath='RiboSET_ext_genomic_primary_ext.bed', 
        df=RiboSET_EXT_cds_cov_nonzero[['tr_id', 'strand', 
                                'global_coo_primary']])

In [10]:
!bedtools sort -i PhyloSET_genomic_primary_ext.bed > PhyloSET_genomic_primary_ext_sorted.bed

!bedtools sort -i RiboSET_ext_genomic_primary_ext.bed > RiboSET_ext_genomic_primary_ext_sorted.bed

In [11]:
!bedtools intersect -a A -b B -wao -s > C

Error: Unable to open file A. Exiting.


# Genomic coo of RiboSET_ext predictions 

In [12]:
RiboSET_EXT_cds_cov_nonzero[['tr_id', 'Start_ext', 'Stop_ext']].to_csv('local_coo_RiboSET_ext.txt',
                                                                      sep='\t', index=False)

In [13]:
def prepare_global_coo(path_to_file_with_global_coo, colname):
    global_coo_g25 = pd.read_csv(path_to_file_with_global_coo, sep='\t')
    global_coo_g25_f = global_coo_g25[global_coo_g25['hit'] == True]
    global_coo_g25_f = global_coo_g25_f.sort_values(by=['seqnames', 'start'])
    global_coo_g25_f['global_coo'] = global_coo_g25_f['seqnames']+':'+global_coo_g25_f['start'].astype(str) +'-'+global_coo_g25_f['end'].astype(str)
    global_coo_g25_f = global_coo_g25_f[['group_name', 'strand', 'global_coo']].groupby(['group_name', 'strand']).agg('+'.join).reset_index()
    global_coo_g25_f.columns = ['tr_id', 'strand', colname]
    return global_coo_g25_f

In [14]:
ext_global = prepare_global_coo(path_to_file_with_global_coo='global_coo_RiboSET_ext.txt',
                                   colname = 'global_coo_ext')

In [15]:
ext_global.shape[0]

3451

In [16]:
RiboSET_EXT_COO = RiboSET_EXT_cds_cov_nonzero[['tr_id', 'gene', 'Riboseq_Summary']].merge(ext_global, on='tr_id', how='inner')

In [17]:
get_bed(filepath='RiboSET_ext_genomic_EXT.bed', 
        df=RiboSET_EXT_COO[['tr_id', 'strand', 
                                'global_coo_ext']])

In [47]:
!bedtools sort -i RiboSET_ext_genomic_EXT.bed > RiboSET_ext_genomic_EXT_sorted.bed

# ClinVar https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/ 

In [49]:
# 

In [18]:
clinvar = pd.read_csv('clinvar.vcf', sep='\t', comment='#', header=None)

clinvar['chr'] = 'chr'+clinvar[0].astype(str)

clinvar.columns = ['chr_', 'start_pos', 'clinvar_ID', 'ref', 'alt', 'qual', 'filter', 'info', 'chr']

# filter normal chromosomes 
clinvar_f = clinvar[clinvar['chr'].isin(['chr'+str(i) for i in range(1, 23)]+['chrY', 'chrX', 'chrM'])]

print (clinvar.shape, clinvar_f.shape)

clinvar_f = clinvar_f[clinvar_f['info'].str.contains('CLNSIG=')]

print (clinvar.shape, clinvar_f.shape)

/home/alla/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1479076, 9) (1476193, 9)
(1479076, 9) (1475557, 9)


In [ ]:
# Included variants do not have an associated disease (CLNDN, CLNDISDB) or a clinical significance (CLNSIG)
# Instead there are three tags are specific to the included variants:
# CLNDNINCL -  ClinVar's preferred disease name for an interpretation for a haplotype or genotype that includes this variant
# CLNDISDBINCL - database name and identifier for the disease name for an interpretation for a haplotype or genotype that includes this variant
# CLNSIGINCL -the clinical significance of a haplotype or genotype that  includes this variant.

In [19]:
clinvar_f['Clin_sign'] = [x.split('CLNSIG=')[1].split(';')[0] for x in clinvar_f['info'].tolist()]

clinvar_f['location_type'] = [x.split('|')[1].split(';')[0].split(',')[0] if x.find('|') != -1 else None for x in clinvar_f['info'].tolist()]

clinvar_f['review_status'] = [x.split('CLNREVSTAT')[1].split(';')[0] if x.find('|') != -1 else None for x in clinvar_f['info'].tolist()]

#review_status_filter = ['=criteria_provided,_single_submitter', '=criteria_provided,_multiple_submitters,_no_conflicts', 
#                       '=reviewed_by_expert_panel', '=practice_guideline']

In [20]:
clinvar['ann'] = 'ref:'+clinvar['ref']+';alt:'+clinvar['alt']+';info:'+clinvar['info']

clinvar['len_alt'] = clinvar['alt'].str.len()

clinvar['stop_pos'] = clinvar['start_pos'] + clinvar['len_alt'] - 1

clinvar[['chr', 'start_pos', 'stop_pos', 'ann']].to_csv('tmp_res/clinvarVCF.bed', sep='\t', index=False, header=None)

# long variants 

In [21]:
tmp = clinvar[['chr', 'start_pos', 'stop_pos', 'ann']]

tmp[tmp['stop_pos'] - tmp['start_pos'] > 1]

,chr,start_pos,stop_pos,ann
31,chr1,930222,930229,ref:GAACTC;alt:TTCTTCTG;info:ALLELEID=987768;C...
116,chr1,935845,935848,ref:A;alt:AGCC;info:ALLELEID=1352540;CLNDISDB=...
124,chr1,935866,935885,ref:G;alt:GGCAGCCTGGAGATTGGCCT;info:ALLELEID=1...
192,chr1,939398,939446,ref:G;alt:GCCTCCCCAGCCACGGTGAGGACCCACCCTGGCATG...
193,chr1,939398,939494,ref:G;alt:GCCTCCCCAGCCACGGTGAGGACCCACCCTGGCATG...
...,...,...,...,...
1477751,chrMT,9273,9276,ref:T;alt:TATC;info:ALLELEID=590157;CLNDISDB=H...
1477762,chrMT,9311,9314,ref:T;alt:TGCA;info:ALLELEID=581839;CLNDISDB=H...
1477785,chrMT,9429,9432,ref:G;alt:GCCC;info:ALLELEID=590155;CLNDISDB=H...
1477789,chrMT,9441,9444,ref:C;alt:CTTT;info:ALLELEID=590156;CLNDISDB=H...


In [ ]:
# RiboSET_ext_genomic_EXT_sorted.bed

# PhyloSET_genomic_primary_ext_sorted.bed

# RiboSET_ext_genomic_primary_ext_sorted.bed

In [22]:
! bedtools intersect -wao -a RiboSET_ext_genomic_EXT_sorted.bed -b tmp_res/clinvarVCF.bed > tmp_res/RiboSET_EXT_pred_start_codon_clinvar.bed

# Parse ovlp between PRED RIBOSET_EXT

In [58]:
res_start = pd.read_csv('tmp_res/RiboSET_EXT_pred_start_codon_clinvar.bed', sep='\t', header=None)

#res_start.columns = ['chr_codon', 'start_codon', 'stop_codon', 'tr_id', 'chr_var', 'start_var', 'stop_var',
#                   'ann_var', '-1']

res_start[res_start[6] != '.']

,0,1,2,3,4,5,6,7,8,9,10
0,chr1,1013520,1013574,ENST00000379389.4,1,+,chr1,1013541,1013541,ref:T;alt:C;info:ALLELEID=1175251;CLNDISDB=MON...,0
11,chr1,2019358,2019424,ENST00000378585.5,1,+,chr1,2019420,2019420,ref:G;alt:A;info:ALLELEID=1306100;CLNDISDB=Med...,0
21,chr1,6466107,6466158,ENST00000377782.7,1,-,chr1,6466123,6466123,ref:G;alt:A;info:AF_ESP=0.16589;AF_EXAC=0.1693...,0
22,chr1,6466107,6466158,ENST00000377782.7,1,-,chr1,6466142,6466142,ref:CCGTGCTCT;alt:C;info:ALLELEID=281645;CLNDI...,0
23,chr1,6466107,6466158,ENST00000377782.7,1,-,chr1,6466158,6466158,ref:T;alt:C;info:ALLELEID=864719;CLNDISDB=MOND...,-1
...,...,...,...,...,...,...,...,...,...,...,...
5185,chrX,147912111,147912180,ENST00000370475.8,1,+,chrX,147912049,147912142,ref:C;alt:CGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGC...,31
5186,chrX,147912111,147912180,ENST00000370475.8,1,+,chrX,147912049,147912145,ref:C;alt:CGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGC...,34
5187,chrX,147912111,147912180,ENST00000370475.8,1,+,chrX,147912049,147912148,ref:C;alt:CGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGC...,37
5188,chrX,147912111,147912180,ENST00000370475.8,1,+,chrX,147912049,147912151,ref:C;alt:CGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGC...,40


In [59]:
for el in res_start.to_numpy():
    info = el[-2]
    tr_id = el[3]
    if info.find('CLNSIG=') != -1:
        Cl_sign = info.split('CLNSIG=')[1].split(';')[0]
        
        #if (Cl_sign.find('Pathogenic') != -1) | (Cl_sign.find('pathogen') != -1):
        print (tr_id, Cl_sign, info.split(';')[0], info.split(';')[1])
        print (info)
        print ()

ENST00000379389.4 Benign ref:T alt:C
ref:T;alt:C;info:ALLELEID=1175251;CLNDISDB=MONDO:MONDO:0014502,MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Mendelian_susceptibility_to_mycobacterial_diseases_due_to_complete_ISG15_deficiency;CLNHGVS=NC_000001.11:g.1013541T>C;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001623|5_prime_UTR_variant;ORIGIN=1

ENST00000378585.5 Uncertain_significance ref:G alt:A
ref:G;alt:A;info:ALLELEID=1306100;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.11:g.2019420G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=GABRD:2563;MC=SO:0001623|5_prime_UTR_variant;ORIGIN=1

ENST00000377782.7 Benign ref:G alt:A
ref:G;alt:A;info:AF_ESP=0.16589;AF_EXAC=0.16938;AF_TGP=0.20228;ALLELEID=281043;CLNDISDB=MONDO:MONDO:0012608,MedGen:C1970211,OMIM:611067,Orphanet:ORPHA206580;

ENST00000614302.4 Uncertain_significance ref:T alt:A
ref:T;alt:A;info:ALLELEID=979820;CLNDISDB=Human_Phenotype_Ontology:HP:0002912,Human_Phenotype_Ontology:HP:0003123,Human_Phenotype_Ontology:HP:0008295,MONDO:MONDO:0002012,MeSH:C537358,MedGen:C0268583,SNOMED_CT:42393006;CLNDN=Methylmalonic_acidemia;CLNHGVS=NC_000016.10:g.89100674T>A;CLNREVSTAT=no_assertion_criteria_provided;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ACSF3:197322;MC=SO:0001619|non-coding_transcript_variant,SO:0001623|5_prime_UTR_variant,SO:0001627|intron_variant;ORIGIN=1;RS=1210097201

ENST00000614302.4 Pathogenic/Likely_pathogenic ref:A alt:G
ref:A;alt:G;info:AF_ESP=0.00016;AF_EXAC=0.00007;ALLELEID=200302;CLNDISDB=MONDO:MONDO:0013661,MedGen:C3280314,OMIM:614265,Orphanet:ORPHA289504|MedGen:CN517202;CLNDN=Combined_malonic_and_methylmalonic_acidemia|not_provided;CLNHGVS=NC_000016.10:g.89100682A>G;CLNREVSTAT=criteria_provided,_multiple_submitters,_no_conflicts;CLNSIG=Pathogeni

ref:A;alt:T;info:ALLELEID=765235;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000005.10:g.80654775A>T;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=DHFR:1719|MSH3:4437;MC=SO:0001619|non-coding_transcript_variant,SO:0001819|synonymous_variant,SO:0001623|5_prime_UTR_variant;ORIGIN=1;RS=976119718

ENST00000439211.6 Uncertain_significance ref:G alt:A
ref:G;alt:A;info:ALLELEID=924118;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000005.10:g.80654776G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=DHFR:1719|MSH3:4437;MC=SO:0001583|missense_variant,SO:0001619|non-coding_transcript_variant,SO:0001623|5_prime_UTR_variant;ORIGIN=1;RS=1176057750

ENST00000439211.6 Uncertain_significance ref:C alt:G
ref:C;alt:G;info:ALLELEID=830978;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000005.10:g.80654777C>

In [60]:
genes = []
variants = 0

li = []

for el in res_start.to_numpy():
    info = el[-2]
    tr_id = el[3]
    if info.find('CLNSIG=') != -1:
        Cl_sign = info.split('CLNSIG=')[1].split(';')[0]
        
        if (Cl_sign.find('Pathogenic') != -1) | (Cl_sign.find('pathogen') != -1):
            print (tr_id, RiboSET_EXT_cds_cov_nonzero[RiboSET_EXT_cds_cov_nonzero['tr_id'] == tr_id].iloc[0].gene, Cl_sign, info.split(';')[0], info.split(';')[1])
            genes.append(RiboSET_EXT_cds_cov_nonzero[RiboSET_EXT_cds_cov_nonzero['tr_id'] == tr_id].iloc[0].gene)
            variants += 1
            #print (list(el[[6,7,8]]))
            
            coo_ = [str(x) for x in list(el[[6,7,8, 5]])]
            
            coo = coo_[0]+':'+coo_[1]+'-'+coo_[2]+';'+coo_[3]
            li.append([tr_id, RiboSET_EXT_cds_cov_nonzero[RiboSET_EXT_cds_cov_nonzero['tr_id'] == tr_id].iloc[0].gene, info.split(';')[0], info.split(';')[1], coo, Cl_sign])

ENST00000302118.5 PCSK9 Conflicting_interpretations_of_pathogenicity ref:C alt:T
ENST00000302118.5 PCSK9 Conflicting_interpretations_of_pathogenicity ref:T alt:C
ENST00000621517.1 GJA5 Conflicting_interpretations_of_pathogenicity ref:C alt:T
ENST00000271715.6 POGZ Likely_pathogenic ref:T alt:C
ENST00000367975.6 SDHC Pathogenic ref:A alt:G
ENST00000367975.6 SDHC Pathogenic ref:A alt:T
ENST00000236137.9 SLC19A2 Conflicting_interpretations_of_pathogenicity ref:G alt:C
ENST00000236137.9 SLC19A2 Conflicting_interpretations_of_pathogenicity ref:G alt:A
ENST00000367535.7 NCF2 Pathogenic ref:T alt:A
ENST00000366574.6 RYR2 Conflicting_interpretations_of_pathogenicity ref:G alt:T
ENST00000623527.3 CDHR1 Conflicting_interpretations_of_pathogenicity ref:A alt:G
ENST00000623527.3 CDHR1 Pathogenic ref:A alt:T
ENST00000260731.4 KIF11 Pathogenic ref:CAT alt:AA
ENST00000260731.4 KIF11 Likely_pathogenic ref:A alt:G
ENST00000369085.7 BAG3 Conflicting_interpretations_of_pathogenicity ref:G alt:A
ENST00000

ENST00000389054.7 DIAPH1 Conflicting_interpretations_of_pathogenicity ref:C alt:A
ENST00000372389.7 POLR1C Conflicting_interpretations_of_pathogenicity ref:G alt:A
ENST00000355773.5 SLC17A5 Pathogenic ref:T alt:A
ENST00000355773.5 SLC17A5 Likely_pathogenic ref:T alt:C
ENST00000314641.9 BVES Pathogenic ref:T alt:C
ENST00000230124.7 FIG4 Pathogenic ref:TGCCGCCGCCATGCCCACG alt:T
ENST00000230124.7 FIG4 Pathogenic ref:A alt:C
ENST00000357525.5 PLN Conflicting_interpretations_of_pathogenicity ref:C alt:T
ENST00000419517.2 CEP85L Likely_pathogenic ref:T alt:A
ENST00000366898.5 PARK2 Likely_pathogenic ref:T alt:C
ENST00000342771.8 AUTS2 Pathogenic ref:A alt:T
ENST00000248553.6 HSPB1 Conflicting_interpretations_of_pathogenicity ref:G alt:A
ENST00000248553.6 HSPB1 Conflicting_interpretations_of_pathogenicity ref:C alt:T
ENST00000343529.9 RELN Conflicting_interpretations_of_pathogenicity ref:T alt:TGCC
ENST00000343529.9 RELN Conflicting_interpretations_of_pathogenicity ref:T alt:TGCCGCC
ENST00000

In [61]:
res_start

,0,1,2,3,4,5,6,7,8,9,10
0,chr1,1013520,1013574,ENST00000379389.4,1,+,chr1,1013541,1013541,ref:T;alt:C;info:ALLELEID=1175251;CLNDISDB=MON...,0
1,chr1,1267992,1267992,ENST00000349431.10,1,-,.,-1,-1,.,0
2,chr1,1273666,1273713,ENST00000349431.10,1,-,.,-1,-1,.,0
3,chr1,1307815,1307845,ENST00000354700.9,1,-,.,-1,-1,.,0
4,chr1,1374756,1375053,ENST00000338370.7,1,-,.,-1,-1,.,0
...,...,...,...,...,...,...,...,...,...,...,...
5208,chrX,155070694,155070719,ENST00000369484.7,1,-,.,-1,-1,.,0
5209,chrY,2841580,2841625,ENST00000250784.12,1,+,.,-1,-1,.,0
5210,chrY,12708667,12708693,ENST00000338981.7,1,+,.,-1,-1,.,0
5211,chrY,12709388,12709448,ENST00000338981.7,1,+,.,-1,-1,.,0


In [62]:
pd.DataFrame(li, columns = ['tr_id', 'gene', 'ref', 'alt', 'coo', 'type']).to_csv('Supplementary_table_9_clinvar.txt', sep='\t', index=False)

In [ ]:
Conflicting_interpretations_of_pathogenicity / Pathogenic

# Ovlp with PhyloSET primary ext

In [68]:
! bedtools intersect -wao -a PhyloSET_genomic_primary_ext_sorted.bed -b tmp_res/clinvarVCF.bed > tmp_res/PhyloSET_pred_start_codon_clinvar.bed

In [54]:
res_start = pd.read_csv('tmp_res/PhyloSET_pred_start_codon_clinvar.bed', sep='\t', header=None)

#res_start.columns = ['chr_codon', 'start_codon', 'stop_codon', 'tr_id', 'chr_var', 'start_var', 'stop_var',
#                   'ann_var', '-1']

res_start[res_start[6] != '.']

,0,1,2,3,4,5,6,7,8,9,10
30,chr16,79599903,79600061,ENST00000326043.4,1,-,chr16,79599908,79599908,ref:TGCC;alt:T;info:ALLELEID=255890;CLNDISDB=M...,0
31,chr16,79599903,79600061,ENST00000326043.4,1,-,chr16,79599908,79599908,ref:TGCCGCCGCC;alt:T;info:ALLELEID=255891;CLND...,0
32,chr16,79599903,79600061,ENST00000326043.4,1,-,chr16,79599908,79599908,ref:TGCCGCCGCCGCC;alt:T;info:ALLELEID=791647;C...,0
33,chr16,79599903,79600061,ENST00000326043.4,1,-,chr16,79599990,79599990,ref:G;alt:A;info:ALLELEID=1206789;CLNDISDB=Med...,0
34,chr16,79599903,79600061,ENST00000326043.4,1,-,chr16,79600054,79600054,ref:G;alt:A;info:ALLELEID=1254414;CLNDISDB=Med...,0
35,chr16,79599903,79600061,ENST00000393350.1,1,-,chr16,79599908,79599908,ref:TGCC;alt:T;info:ALLELEID=255890;CLNDISDB=M...,0
36,chr16,79599903,79600061,ENST00000393350.1,1,-,chr16,79599908,79599908,ref:TGCCGCCGCC;alt:T;info:ALLELEID=255891;CLND...,0
37,chr16,79599903,79600061,ENST00000393350.1,1,-,chr16,79599908,79599908,ref:TGCCGCCGCCGCC;alt:T;info:ALLELEID=791647;C...,0
38,chr16,79599903,79600061,ENST00000393350.1,1,-,chr16,79599990,79599990,ref:G;alt:A;info:ALLELEID=1206789;CLNDISDB=Med...,0
39,chr16,79599903,79600061,ENST00000393350.1,1,-,chr16,79600054,79600054,ref:G;alt:A;info:ALLELEID=1254414;CLNDISDB=Med...,0


In [56]:
genes = []
variants = 0

for el in res_start.to_numpy():
    info = el[-2]
    tr_id = el[3]
    if info.find('CLNSIG=') != -1:
        Cl_sign = info.split('CLNSIG=')[1].split(';')[0]
        
        if (Cl_sign.find('Pathogenic') != -1) | (Cl_sign.find('pathogen') != -1):
            coo_ = [str(x) for x in list(el[[6,7,8, 5]])]
            
            coo = coo_[0]+':'+coo_[1]+'-'+coo_[2]+';'+coo_[3]
            print (tr_id+'\t'+PhyloSET[PhyloSET['tr_id'] == tr_id].iloc[0].gene+'\t'+Cl_sign+'\t'+info.split(';')[0]+'\t'+info.split(';')[1]+'\t'+coo)
            genes.append(PhyloSET[PhyloSET['tr_id'] == tr_id].iloc[0].gene)
            variants += 1

ENST00000374375.1	GDF5OS	Pathogenic/Likely_pathogenic	ref:A	alt:G	chr20:35434093-35434093;+
ENST00000374375.1	GDF5OS	Pathogenic	ref:GC	alt:AA	chr20:35434101-35434102;+
ENST00000374375.1	GDF5OS	Pathogenic	ref:C	alt:A	chr20:35434102-35434102;+
ENST00000374375.1	GDF5OS	Pathogenic	ref:G	alt:T	chr20:35434109-35434109;+


In [72]:
len(set(genes)), variants

(1, 4)

In [ ]:
1 gene with 4 variants 

# Scan for domains 

Short sequences (<80 aa) are unlikely to have any signature matches, 

In [162]:
f = open('RiboSET_EXT_aa_100.fasta', 'w')

d={}

for el in RiboSET_EXT_cds_cov_nonzero[['tr_id', 'Start_ext', 'Stop_ext', 'gene']][0:100].to_numpy():
    tr_id = el[0]
    start = el[1]-1
    stop = el[2] #nt of ext in transcript 
    gene = el[-1]
    
    pos_of_ext_stop_in_aa = (stop - start)//3
    
    cds_stop_pos = metadata_pc_g25[metadata_pc_g25['tr_id'] == tr_id].iloc[0].cds_stop_pos
    
    seq = metadata_pc_g25[metadata_pc_g25['tr_id'] == tr_id].iloc[0].transcript_seq[start:cds_stop_pos]
    
    aa = str('M'+Seq(seq).translate()[1:-1])
    
    #if len(aa) >= 80:
    f.write('>'+gene+'_'+tr_id+'_'+str(stop)+'\n')
    f.write(aa+'\n')
    
    d[tr_id] = aa+'_'+str(pos_of_ext_stop_in_aa)
    
    
f.close()


In [163]:
!wc -l RiboSET_EXT_aa_100.fasta

200 RiboSET_EXT_aa_100.fasta


In [96]:
/2

3451.0

In [107]:
!head RiboSET_EXT_aa_500.fasta

>ASPH_ENST00000379454.8_189
MKEVRQPSPAPADRAMAQRKNAKSSGNSSSSGSGSGSTSAGSSSPGARRETKHGGHKNGRKGGLSGTSFFTWFMVIALLGVWTSVAVVWFDLVDYEEVLGKLGIYDADGDGDFDVDDAKVLLGLKERSTSEPAVPPEEAEPHTEPEEQVPVEAEPQNIEDEAKEQIQSLLHEMVHAEHVEGEDLQQEDGPTGEPQQEDDEFLMATDVDDRFETLEPEVSHEETEHSYHVEETVSQDCNQDMEEMMSEQENPDSSEPVVEDERLHHDTDDVTYQVYEEQAVYEPLENEGIEITEVTAPPEDNPVEDSQVIVEEVSIFPVEEQQEVPPETNRKTDDPEQKAKVKKKKPKLLNKFDKTIKAELDAAEKLRKRGKIEEAVNAFKELVRKYPQSPRARYGKAQCEDDLAEKRRSNEVLRGAIETYQEVASLPDVPADLLKLSLKRRSDRQQFLGHMRGSLLTLQRLVQLFPNDTSLKNDLGVGYLLIGDNDNAKKVYEEVLSVTPNDGFAKVHYGFILKAQNKIAESIPYLKEGIESGDPGTDDGRFYFHLGDAMQRVGNKEAYKWYELGHKRGHFASVWQRSLYNVNGLKAQPWWTPKETGYTELVKSLERNWKLIRDEGLAVMDKAKGLFLPEDENLREKGDWSQFTLWQQGRRNENACKGAPKTCTLLEKFPETTGCRRGQIKYSIMHPGTHVWPHTGPTNCRLRMHLGLVIPKEGCKIRCANETKTWEEGKVLIFDDSFEHEVWQDASSFRLIFIVDVWHPELTPQQRRSLPAI
>HSP90AA1_ENST00000216281.12_207
MAVPLVLCGHLAKMPEETQTQDQPMEEEEVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISNSSDALDKIRYESLTDPSKLDSGKELHINLIPNKQDRTLTIVDTGIGMTKADLINNLGTIAKSGTKAFMEALQAGADISMIGQFGVGFYSAYLVAEKVTVIT

# Parse gtf from interpro

In [114]:
!tail -n 20 interpro_test/iprscan5-R20221012-100927-0792-79930168-p1m.gff.txt

>match$90_121_177
LFVGGLKGDVAEGDLIEHFSQFGTVEKAEIIADKQSGKKRGFGFVYFQNHDAADKAA
>match$91_115_202
HAKVKKLFVGGLKGDVAEGDLIEHFSQFGTVEKAEIIADKQSGKKRGFGFVYFQNHDAAD
KAAVVKFHPIQGHRVEVKKAVPKEDIYS
>match$92_119_196
KKLFVGGLKGDVAEGDLIEHFSQFGTVEKAEIIADKQSGKKRGFGFVYFQNHDAADKAAV
VKFHPIQGHRVEVKKAVP
>match$93_120_199
KLFVGGLKGDVAEGDLIEHFSQFGTVEKAEIIADKQSGKKRGFGFVYFQNHDAADKAAVV
KFHPIQGHRVEVKKAVPKED
>match$94_22_322
MENSQLCKLFIGGLNVQTSESGLRGHFEAFGTLTDCVVVVNPQTKRSRCFGFVTYSNVEE
ADAAMAASPHAVDGNTVELKRAVSREDSARPGAHAKVKKLFVGGLKGDVAEGDLIEHFSQ
FGTVEKAEIIADKQSGKKRGFGFVYFQNHDAADKAAVVKFHPIQGHRVEVKKAVPKEDIY
SGGGGGGSRSSRGGRGGRGRGGGRDQNGLSKGGGGGYNSYGGYGGGGGGGYNAYGGGGGG
SSYGGSDYGNGFGGFGSYSQHQSSYGPMKSGGGGGGGGSSWGGRSNSGPYRGGYGGGGGY
G
>match$95_195_235
VPKEDIYSGGGGGGSRSSRGGRGGRGRGGGRDQNGLSKGGG


In [166]:
!head iprscan5-R20221013-062400-0292-16358696-p1m.gff.txt

##gff-version 3
##feature-ontology http://song.cvs.sourceforge.net/viewvc/song/ontology/sofa.obo?revision=1.269
##interproscan-version 5.57-90.0
##sequence-region RAD1_ENST00000382038.6_1421 1 319
RAD1_ENST00000382038.6_1421	.	polypeptide	1	319	.	+	.	ID=RAD1_ENST00000382038.6_1421;md5=fcef0bff33ee96c2f4aa6e5a6eb2072f
RAD1_ENST00000382038.6_1421	PRINTS	protein_match	78	91	8.4E-48	+	.	date=13-10-2022;Target=RAD1_ENST00000382038.6_1421 78 91;Ontology_term="GO:0000077","GO:0005634";ID=match$1_78_91;signature_desc=Rad1 repair protein signature;Name=PR01246;status=T;Dbxref="InterPro:IPR003011","Reactome:R-CEL-176187","Reactome:R-HSA-176187","Reactome:R-HSA-5685938","Reactome:R-HSA-5693607","Reactome:R-HSA-5693616","Reactome:R-HSA-6804756","Reactome:R-HSA-69473","Reactome:R-HSA-9709570","Reactome:R-MMU-176187","Reactome:R-MMU-5685938","Reactome:R-MMU-5693607","Reactome:R-MMU-6804756","Reactome:R-MMU-69473","Reactome:R-SPO-176187"
RAD1_ENST00000382038.6_1421	PRINTS	protein_match	117	133	8.4E-4

In [171]:
gtf = pd.read_csv('iprscan5-R20221013-062400-0292-16358696-p1m.gff.txt', 
                  sep='\t', comment='#', header=None)
gtf

,0,1,2,3,4,5,6,7,8
0,RAD1_ENST00000382038.6_1421,.,polypeptide,1.0,319.0,.,+,.,ID=RAD1_ENST00000382038.6_1421;md5=fcef0bff33e...
1,RAD1_ENST00000382038.6_1421,PRINTS,protein_match,78.0,91.0,8.4E-48,+,.,date=13-10-2022;Target=RAD1_ENST00000382038.6_...
2,RAD1_ENST00000382038.6_1421,PRINTS,protein_match,117.0,133.0,8.4E-48,+,.,date=13-10-2022;Target=RAD1_ENST00000382038.6_...
3,RAD1_ENST00000382038.6_1421,PRINTS,protein_match,240.0,253.0,8.4E-48,+,.,date=13-10-2022;Target=RAD1_ENST00000382038.6_...
4,RAD1_ENST00000382038.6_1421,PRINTS,protein_match,260.0,278.0,8.4E-48,+,.,date=13-10-2022;Target=RAD1_ENST00000382038.6_...
...,...,...,...,...,...,...,...,...,...
9672,GKQLGKLLLSTLTLLSKKLNCYKITLECLPQNVGFYKKFGY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9673,>match$1691_61_230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9674,LKEVDWSQNTATFSPAISPTHPGEGLVLRPLCTADLNRGFFKVLGQ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9675,SFEHMKKSGDYYVTVVEDVTLGQIVATATLIIEHKFIHSCAKRGRV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
for el in gtf[gtf[1] == 'Pfam'][[0, 3, 4, 8]].to_numpy():
    tr_id = el[0].split('_')[1]
    gene = el[0].split('_')[0]
    end_of_ext_aa = el[0].split('_')[2]
    start = el[1]
    stop = el[2] 
    domain_desc = el[3]
    
    domain_aa = d[tr_id].split('_')[0]#[start:stop]
    
    stop_pos_of_ext_in_aa = d[tr_id].split('_')[1]
    
    if int(stop_pos_of_ext_in_aa)>=start:
        print(tr_id, 'end_of_ext:', stop_pos_of_ext_in_aa, 'start_of_pred_domain: ', start)

In [174]:
start

54.0

In [175]:
stop_pos_of_ext_in_aa

'37'